In [2]:
# this module finds the coordinates of a given depth contour
from __future__ import division, print_function

import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import itertools

from salishsea_tools import (
    nc_tools,
    viz_tools,
)

In [3]:
%matplotlib inline

In [4]:
def iso_bathy(bathy,ist,jst,direction,ijend,thres,depth):
    #Starting at (ist,jst) this function computes the gridpoints of the depth contour by iterating through points until ijend.
    #At each iteration it calculates the maximum depth in a neighborhood of (i,j)
    #bathy: an array of the bathymetry
    #ist: an int that defines the starting i grid point
    #jst: an int that defines the starting j grid point
    #direction: a string either 'i' or 'j' indicating the direction of the iterations
    #thres: an int defining the size of the neighbourhood.
    
    #returns thalweg: an array contatining the [j,i] thalweg gridpoints.
    
    if direction is 'i':
    
        #thres defines the search distance in the i-direction.
        if ist<ijend:
            thalweg=np.zeros((ijend-ist,2))
            loop_start=ist+1; loop_end=ijend
            forward=1; backward=0
        else:
            thalweg=np.zeros((ist-ijend,2)) 
            loop_start=0; loop_end=ist-ijend-1;
            forward=0; backward=1;
    
        #find index of first maximum
        bathmax=bathy[jst-thres:jst+thres,ist].max(); 
        ind= np.where(bathy[:,ist]==bathmax)
        ls=0
        while ind[0][ls]< jst-thres:
            ls=ls+1
        j=ind[0][ls]
        arr_ind=0*forward + (ist-ijend-1)*backward;  
        thalweg[arr_ind,0]=j
        thalweg[arr_ind,1]=ist

        #for loop for finding the thalweg
        for k in range (loop_start,loop_end):
            loop_ind = k*forward + (ist-k)*backward;
            bathmax=bathy[j-thres:j+thres,loop_ind].max();
            ind= np.where(bathy[:,loop_ind]==bathmax)
            ls=0
            while ind[0][ls]< j-thres:
                ls=ls+1
            j=ind[0][ls]
            arr_ind=(k-ist)*forward + (loop_end-k-1)*backward;  
            thalweg[arr_ind,0]=j
            thalweg[arr_ind,1]=loop_ind;
            
    elif direction is 'j':
    
        #thres defines the search distance in the j-direction.
        if jst<ijend:
            thalweg=np.zeros((ijend-jst,2))
            loop_start=jst+1; loop_end=ijend
            forward=1; backward=0
        else:
            thalweg=np.zeros((jst-ijend,2)) 
            loop_start=0; loop_end=jst-ijend-1;
            forward=0; backward=1;
    
        #find index of first maximum
        bathmax=bathy[jst,ist-thres:ist+thres].max(); 
        ind= np.where(bathy[jst,:]==bathmax)
        ls=0
        while ind[0][ls]< ist-thres:
            ls=ls+1
        i=ind[0][ls]
        arr_ind=0*forward + (jst-ijend-1)*backward;  
        thalweg[arr_ind,0]=jst
        thalweg[arr_ind,1]=i

        #for loop for finding the thalweg
        for k in range (loop_start,loop_end):
            loop_ind = k*forward + (jst-k)*backward;
            bathmax=bathy[loop_ind,i-thres:i+thres].max();
            ind= np.where(bathy[loop_ind,:]==bathmax)
            ls=0
            while ind[0][ls]< i-thres:
                ls=ls+1
            i=ind[0][ls]
            arr_ind=(k-jst)*forward + (loop_end-k-1)*backward;  
            thalweg[arr_ind,0]=loop_ind
            thalweg[arr_ind,1]=i;
            
    else:
        return 'Error'
    
    return thalweg